In [1]:
import cv2
import dlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import tensorflow



ModuleNotFoundError: No module named 'dlib'

In [2]:
cascade = cv2.CascadeClassifier(r'C:\Users\Hardik Arora\Desktop\personal projects\haar_cascades files\haarcascade_frontalface_default.xml')

predictor = dlib.shape_predictor(r'C:\Users\Hardik Arora\Desktop\personal projects\model_h5_and_pickle_files\shape_predictor_68_face_landmarks.dat')

FACIAL_LANDMARKS_ID={"mouth":(48,68),"right_eyebrow":(17,22),"left_eyebrow":(22,27),"right_eye":(36,42),"left_eye":(42,48),"nose":(27,35),"jaw":(0,17)}



In [3]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype="int")
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

In [45]:
class FaceAligner:
    def __init__(self,predictor,desired_left_eye=(0.35,0.35),samplewidth=300,sampleheight=None):
        self.predictor=predictor
        self.desired_left_eye=desired_left_eye
        self.samplewidth=samplewidth
        self.sampleheight=sampleheight
        if(sampleheight==None):
            self.sampleheight=samplewidth
    def align_face(self,image,gray,rect):
        shape=predictor(gray,rect)
        shape=shape_to_np(shape)
        lstart,lend=FACIAL_LANDMARKS_ID["left_eye"]
        rstart,rend=FACIAL_LANDMARKS_ID["right_eye"]
        left_eye_pts=shape[lstart:lend]
        right_eye_pts=shape[rstart:rend]
            
        left_e_center=left_eye_pts.mean(axis=0).astype("int")
        right_e_center=right_eye_pts.mean(axis=0).astype("int")
        dY=right_e_center[1]-left_e_center[1]
        dX=right_e_center[0]-left_e_center[0]
        angle=np.degrees(np.arctan2(dY,dX))-180

        desired_right_eyex=1-self.desired_left_eye[0]
        dist=np.sqrt(dX**2+dY**2)
        desired_dist=desired_right_eyex-self.desired_left_eye[0]
        desired_dist*=self.samplewidth
        scale=desired_dist/dist

        eye_center=((left_e_center[0]+right_e_center[0])//2,(left_e_center[1]+right_e_center[1])//2)
        M=cv2.getRotationMatrix2D(eye_center,angle,scale)
        tX = self.samplewidth * 0.5
        tY = self.sampleheight * self.desired_left_eye[1]
        M[0, 2] += (tX - eye_center[0])
        M[1, 2] += (tY - eye_center[1])
        (w,h)=(self.samplewidth,self.sampleheight)
        output=cv2.warpAffine(image,M,(w,h),flags=cv2.INTER_CUBIC)

        return output

In [46]:
fa = FaceAligner(predictor, samplewidth=256)


In [55]:
def align_face_new(frame):
    rects=[]
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=cascade.detectMultiScale(gray,scaleFactor=1.05,minSize=(350,350))
    for f in faces:
        (x,y,w,h)=f
        rect=(x,y,w,h)
        rects.append(rect)
    
    rect=list(enumerate(rects))
    for i,r in (rect):
        (x,y,w,h)=r
        rectangle=dlib.rectangle(np.uint32(x).item(),np.uint32(y).item(),np.uint32(x).item()+np.uint32(w).item(),np.uint32(x).item()+np.uint32(h).item())
        output=fa.align_face(frame,gray,rectangle)
        image=frame[y:y+h,x:x+w]
        w,h=image.shape[0:2]
        output=cv2.resize(output,(w,h))
        frame[y:y+h,x:x+w]=output
    return frame

(2, 32, 24, 200, 360, 3)


In [ ]:
img=cv2.imread("sample_image1.jpg")
out=align_face_new(img)
cv2.imshow("output_aligned",out)
cv2.waitKey()
cv2.destroyAllWindows()